In [5]:
import tensorflow as tf
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
def initialize_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]

    #print [str(i.name) for i in not_initialized_vars] # only for testing
    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))
        
    list_of_variables = tf.global_variables()
   
    uninitialized_variables = sess.run(tf.report_uninitialized_variables(list_of_variables))
    
    if len(uninitialized_variables)>0:
        return uninitialized_variables
    else:
        return "all_variables_initialized"

### prepare some input and label for a classification problem

In [2]:
ohe=OneHotEncoder()
ohe.fit(np.array(['A','C','G','T']).reshape(-1,1))

seq=[]
label=[]
for i in range(1000):
    tmp=np.array(random.choices('ACGT',k=100))
    seq.append(ohe.transform(tmp.reshape(-1,1)).toarray())
    label.append(random.choices([0,1],k=1))

seq=np.array(seq)
label=np.array(label).reshape(-1,1)

### train and save the network

In [3]:
tf.reset_default_graph()
with tf.variable_scope(name_or_scope='ref'):
    v1 = tf.placeholder(tf.float32,shape=(None,100,4), name="v1") 
    v2=tf.layers.conv1d(inputs=v1,filters=1,kernel_size=8,strides=1,padding='valid',
                        data_format='channels_last',use_bias=False,name='conv0',activation='relu')
    v3=tf.layers.batch_normalization(inputs=v2,axis=-1,name='bn')
    v4=tf.layers.dropout(inputs=v3,rate=0.05,name='drop')
    v5=tf.layers.flatten(inputs=v4,name='flat')
    v6=tf.layers.dense(inputs=v5,units=1,activation='sigmoid')
    
with tf.variable_scope(name_or_scope='criteria'):
    target_op=tf.placeholder(tf.float32,shape=(None,1),name='target')
    loss=tf.reduce_mean(tf.square(tf.subtract(target_op,v6)),name='loss') 
    opt=tf.train.AdamOptimizer(learning_rate=0.01,
                               name='ref_adam').minimize(loss,
                                                name='ref_opt',var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
                                                        )

saver = tf.train.Saver()

# cpu
#with tf.Session() as sess: 
# gpu
gpu_options=tf.GPUOptions(allow_growth=True)
sess=tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())

for _ in range(10):
    print(_)
    sess.run([loss,opt],feed_dict={v1:seq,target_op:label}) 

saver.save(sess, "./model/model_test")

graph=tf.get_default_graph()
writer = tf.summary.FileWriter("./model/model_graph", graph=sess.graph,filename_suffix='reference')
#print(tf.pl)

W0911 21:59:09.936074  2580 deprecation.py:323] From <ipython-input-3-cb969bea8024>:5: conv1d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
W0911 21:59:09.941073  2580 deprecation.py:506] From c:\users\yynst\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0911 21:59:10.043072  2580 deprecation.py:323] From <ipython-input-3-cb969bea8024>:6: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE

0
1
2
3
4
5
6
7
8
9


### load the modify the network structure, loss, optimizer, and re-train it

In [3]:
tf.reset_default_graph()
# cpu
# sess=tf.Session() 
# gpu
gpu_options=tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
    
saver = tf.train.import_meta_graph("./model/model_test.meta")
saver.restore(sess,tf.train.latest_checkpoint('./model/'))
graph=tf.get_default_graph()


W0911 22:00:45.197487 23448 deprecation.py:323] From c:\users\yynst\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [4]:
for _ in graph.get_operations():
    print(_.name)

ref/v1
ref/conv0/kernel/Initializer/random_uniform/shape
ref/conv0/kernel/Initializer/random_uniform/min
ref/conv0/kernel/Initializer/random_uniform/max
ref/conv0/kernel/Initializer/random_uniform/RandomUniform
ref/conv0/kernel/Initializer/random_uniform/sub
ref/conv0/kernel/Initializer/random_uniform/mul
ref/conv0/kernel/Initializer/random_uniform
ref/conv0/kernel
ref/conv0/kernel/Assign
ref/conv0/kernel/read
ref/conv0/dilation_rate
ref/conv0/conv1d/ExpandDims/dim
ref/conv0/conv1d/ExpandDims
ref/conv0/conv1d/ExpandDims_1/dim
ref/conv0/conv1d/ExpandDims_1
ref/conv0/conv1d
ref/conv0/conv1d/Squeeze
ref/conv0/Relu
ref/bn/gamma/Initializer/ones
ref/bn/gamma
ref/bn/gamma/Assign
ref/bn/gamma/read
ref/bn/beta/Initializer/zeros
ref/bn/beta
ref/bn/beta/Assign
ref/bn/beta/read
ref/bn/moving_mean/Initializer/zeros
ref/bn/moving_mean
ref/bn/moving_mean/Assign
ref/bn/moving_mean/read
ref/bn/moving_variance/Initializer/ones
ref/bn/moving_variance
ref/bn/moving_variance/Assign
ref/bn/moving_variance/

In [16]:
tf.global_variables()

[<tf.Variable 'ref/conv0/kernel:0' shape=(8, 4, 1) dtype=float32_ref>,
 <tf.Variable 'ref/bn/gamma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/beta:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/moving_mean:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/moving_variance:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/dense/kernel:0' shape=(93, 1) dtype=float32_ref>,
 <tf.Variable 'ref/dense/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'ref/conv0/kernel/Adam:0' shape=(8, 4, 1) dtype=float32_ref>,
 <tf.Variable 'ref/conv0/kernel/Adam_1:0' shape=(8, 4, 1) dtype=float32_ref>,
 <tf.Variable 'ref/bn/gamma/Adam:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/gamma/Adam_1:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/beta/Adam:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/beta/Adam_1:0' shape=(1,) dtype=float

In [4]:
with tf.variable_scope(name_or_scope='new_branch'):
    tmp_op=graph.get_tensor_by_name('ref/drop/Identity:0')
    # new ops
    #new_tmp_op=tf.stop_gradient(tmp_op,name='stop_g')
    new_op1=tf.layers.conv1d(inputs=tmp_op,filters=10,kernel_size=5,padding='valid',name='new_conv')
    new_op2=tf.layers.batch_normalization(inputs=new_op1,axis=-1,name='new_bn')
    new_op3=tf.layers.flatten(inputs=new_op2,name='new_flat')
    new_op4=tf.layers.dense(inputs=new_op3,units=1,name='new_dense')

with tf.variable_scope(name_or_scope='new_criteria'):
    new_loss=tf.reduce_mean(tf.square(tf.subtract(graph.get_tensor_by_name('criteria/target:0'),new_op4)),name='new_loss') 
    new_opt=tf.train.AdamOptimizer(learning_rate=0.01,
                 name='new_adam').minimize(new_loss,name='new_opt',
                                 var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='new'))

W0911 22:00:56.037491 23448 deprecation.py:323] From <ipython-input-4-0d98f3637797>:5: conv1d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
W0911 22:00:56.041487 23448 deprecation.py:506] From c:\users\yynst\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0911 22:00:56.137485 23448 deprecation.py:323] From <ipython-input-4-0d98f3637797>:6: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE

### transfer training

In [7]:
# need to only initialize the un-initialized ones
initialize_uninitialized(sess)
#print(graph.get_tensor_by_name('new_conv/kernel:0').eval(session=sess))
# run new training 
for _ in range(10):
        print(_)
        sess.run([new_loss,new_opt],feed_dict={'ref/v1:0':seq,'criteria/target:0':label})

W0911 22:01:46.552486 23448 deprecation.py:323] From c:\users\yynst\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


0
1
2
3
4
5
6
7
8
9


In [8]:
# show new graph
new_graph_def=tf.graph_util.extract_sub_graph(sess.graph_def,['ref/v1','new_criteria/new_opt'])
writer = tf.summary.FileWriter("./model/model_graph", graph=sess.graph,filename_suffix='new')
#writer = tf.summary.FileWriter("./model/model_graph", graph=sess.graph_def,filename_suffix='new')

In [13]:
# get old weight matrix
graph.get_tensor_by_name('ref/conv0/kernel:0').eval(session=sess)

array([[[-0.18694213],
        [ 0.08943529],
        [ 0.28633773],
        [ 0.14845935]],

       [[-0.19800441],
        [ 0.26258618],
        [ 0.0839562 ],
        [ 0.03638361]],

       [[-0.09319929],
        [ 0.07355726],
        [-0.3320895 ],
        [-0.02227289]],

       [[ 0.21407974],
        [-0.22823995],
        [ 0.09404552],
        [ 0.05545748]],

       [[ 0.21332948],
        [-0.03028096],
        [ 0.25843358],
        [-0.30465853]],

       [[-0.04313145],
        [-0.19690534],
        [ 0.12884268],
        [ 0.02714686]],

       [[-0.44813126],
        [-0.00278444],
        [-0.36645076],
        [-0.27480653]],

       [[ 0.16461603],
        [ 0.28126273],
        [-0.01375197],
        [-0.28515607]]], dtype=float32)

In [11]:
# get new weight matrix
graph.get_tensor_by_name('ref/conv0/kernel:0').eval(session=sess)

array([[[-0.18694213],
        [ 0.08943529],
        [ 0.28633773],
        [ 0.14845935]],

       [[-0.19800441],
        [ 0.26258618],
        [ 0.0839562 ],
        [ 0.03638361]],

       [[-0.09319929],
        [ 0.07355726],
        [-0.3320895 ],
        [-0.02227289]],

       [[ 0.21407974],
        [-0.22823995],
        [ 0.09404552],
        [ 0.05545748]],

       [[ 0.21332948],
        [-0.03028096],
        [ 0.25843358],
        [-0.30465853]],

       [[-0.04313145],
        [-0.19690534],
        [ 0.12884268],
        [ 0.02714686]],

       [[-0.44813126],
        [-0.00278444],
        [-0.36645076],
        [-0.27480653]],

       [[ 0.16461603],
        [ 0.28126273],
        [-0.01375197],
        [-0.28515607]]], dtype=float32)

In [14]:
graph.get_tensor_by_name('new_conv/kernel:0').eval(session=sess)

KeyError: "The name 'new_conv/kernel:0' refers to a Tensor which does not exist. The operation, 'new_conv/kernel', does not exist in the graph."

In [10]:
tf.global_variables()

[<tf.Variable 'ref/conv0/kernel:0' shape=(8, 4, 1) dtype=float32_ref>,
 <tf.Variable 'ref/bn/gamma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/beta:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/moving_mean:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/moving_variance:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/dense/kernel:0' shape=(93, 1) dtype=float32_ref>,
 <tf.Variable 'ref/dense/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'ref/conv0/kernel/Adam:0' shape=(8, 4, 1) dtype=float32_ref>,
 <tf.Variable 'ref/conv0/kernel/Adam_1:0' shape=(8, 4, 1) dtype=float32_ref>,
 <tf.Variable 'ref/bn/gamma/Adam:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/gamma/Adam_1:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/beta/Adam:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'ref/bn/beta/Adam_1:0' shape=(1,) dtype=float

In [33]:
tf.reset_default_graph()
with tf.Session() as sess: 
    saver = tf.train.import_meta_graph("./model/model_test.meta")
    saver.restore(sess,tf.train.latest_checkpoint('./model/'))
    #tf.saved_model.load(sess,tags=[tf.saved_model.tag_constants.SERVING],export_dir='./model/model_ex1')
   
  
    for op in tf.get_default_graph().get_operations():
        print(op.name) 
#     print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
    graph=tf.get_default_graph()
    op=graph.get_tensor_by_name('v4_op:0')
    new_op=tf.divide(op,10)
    new_op=tf.identity(new_op,name='new_v5')
    result = sess.run(new_op, 
                      feed_dict={"v1:0": 12.0, "v2:0": 3.3})  
    
    #print(sess.graph_def)
    # the problem with Basenji is that the input feed_dict is not known with pre-trained model
    #print(sess.graph_def)
    new_graph_def=tf.graph_util.extract_sub_graph(sess.graph_def,['v1','v2','vx','v4_op','new_v5'])  # extract subgraph accessible to destination nodes defined here
    
    # this can still output modified graph, but only for visualization
    writer = tf.summary.FileWriter("./model/model_ex3/model_graph", graph=new_graph_def,filename_suffix='new')
    #writer = tf.summary.FileWriter("./model/model_ex3/model_graph", graph=sess.graph,filename_suffix='old')
    print()
    print(result)
    print()
    for _ in new_graph_def.node:
        print(_.name)
   
    

W0905 16:16:44.739183 11996 writer.py:199] Passing a `GraphDef` to the SummaryWriter is deprecated. Pass a `Graph` object instead, such as `sess.graph`.


v1
v2
Add
vx/initial_value
vx
vx/Assign
vx/read
v4_op
v5/y
v5
save/filename/input
save/filename
save/Const
save/SaveV2/tensor_names
save/SaveV2/shape_and_slices
save/SaveV2
save/control_dependency
save/RestoreV2/tensor_names
save/RestoreV2/shape_and_slices
save/RestoreV2
save/Assign
save/restore_all
init
Add_1
Assign

3.53

v1
v2
Add
vx
vx/read
v4_op
truediv/y
truediv
new_v5


In [25]:
graph=tf.get_default_graph()
print()
print(graph.get_all_collection_keys())
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
#graph.get_operations()
graph.get_tensor_by_name('vx:0')

graph.get_operation_by_name('v5')


['trainable_variables', 'variables']
[<tf.Variable 'vx:0' shape=() dtype=float32_ref>]


<tf.Operation 'v5' type=Pow>